# 📌 id값 추가
- 이후 값 추가 시 ai함수 생성 필요

### ✏️ local DB 연결(TEST용)

In [2]:
import pymongo

conn = pymongo.MongoClient("localhost", 27017)

db_soaeng = conn.soaeng
dbcol_origin = db_soaeng.words
dbcol_words = db_soaeng.voca

# db_bigvoca = conn.bigvoca
# dbcol_voca = db_bigvoca.bigvoca_words

words = list(dbcol_word.find({}).sort("frequency", -1))
for idx, word in enumerate(words):
    word["voca_id"] = idx + 1
    dbcol_word.update_one({"_id": word["_id"]}, {"$set": word})

### ✏️ ChuAnyOne DB 연결

In [26]:
import pymongo
import dotenv
import os

# 환경변수 불러오기
dotenv.load_dotenv(dotenv.find_dotenv())
USER = os.environ["MONGODB_USER"] # MongoDB user
PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port

# DB 연결
client = pymongo.MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)
db_bigvoca = client.bigvoca
dbcol_voca = db_bigvoca.words

# 빈도수 순으로 정렬
words = list(dbcol_voca.find({}).sort("frequency", -1))

# voca_id 추가
for idx, word in enumerate(words):
    word["voca_id"] = idx + 1
    dbcol_voca.update_one({"_id": word["_id"]}, {"$set": word})

# 📌 번역 설정

In [14]:
origin = list(dbcol_origin.find({}).sort("frequency", -1).limit(1500))

# 확인할 단어
diff = []

# 단어장에 들어갈 단어
voca = []
for word in origin:
    # 다르게 번역된 리스트
    if word["korean"] != word["papago"]:
        diff.append(word)
    # 똑같이 번역된 리스트
    else:
        input = {"japanese": word["japanese"], "pronunciation": word["pronunciation"], "korean": word["korean"], "frequency": word["frequency"]}
        voca.append(input)

# 똑같이 번역된 리스트만 단어장에 집어넣기
for word in voca:
    dbcol_words.update_one({"japanese":word["japanese"]}, {"$set": word}, upsert = True)
    
# print(len(diff))
# print(len(voca))
# print(voca)
print("단어 저장 완료")

### ✏️ 데이터값 확인

In [15]:
import pandas as pd

print(pd.DataFrame(diff[:10]))

                        _id  frequency korean pronunciation papago japanese
0  631f757b1750bef094a82161     305364     아니           Nai     없다       無い
1  631f757b1750bef094a8217f     239317    ~이다           Iru     있다       居る
2  631f757b1750bef094a82171     232232     사례          Koto      일        事
3  631f757b1750bef094a82169     221927     좋은           Yoi     좋다       良い
4  631f757b1750bef094a82175     176825   이 되다          Naru  이루어지다       成る
5  631f757b1750bef094a82173     176726    가지다           Aru     있다       有る
6  631f757b1750bef094a8218d     160185      예            Sō     그래       そう
7  631f757b1750bef094a82207     146042     구리            Dō    어떻게       どう
8  631f757b1750bef094a8216f     128920    보내다          Yaru     주다       遣る
9  631f757b1750bef094a821c5     114859     남성          Hito     사람        人


In [ ]:
for word in diff:
    input = {"japanese": word["japanese"], "pronunciation": word["pronunciation"], "korean": word["korean"]+", "+word["papago"], "frequency": word["frequency"]}
    dbcol_words.update_one({"japanese":word["japanese"]}, {"$set": input}, upsert = True)
print("단어 저장 완료")